In [6]:
import torch

In [7]:
with open('wizard_of_oz.txt','r',encoding='utf-8') as f:
    text=f.read()
chars = sorted(set(text))
print(chars)

['\n', ' ', '!', '"', '&', "'", '(', ')', '*', ',', '-', '.', '0', '1', '2', '3', '4', '5', '6', '7', '8', '9', ':', ';', '?', 'A', 'B', 'C', 'D', 'E', 'F', 'G', 'H', 'I', 'J', 'K', 'L', 'M', 'N', 'O', 'P', 'Q', 'R', 'S', 'T', 'U', 'V', 'W', 'X', 'Y', 'Z', '[', ']', '_', 'a', 'b', 'c', 'd', 'e', 'f', 'g', 'h', 'i', 'j', 'k', 'l', 'm', 'n', 'o', 'p', 'q', 'r', 's', 't', 'u', 'v', 'w', 'x', 'y', 'z', '\ufeff']


In [10]:
string_to_int = {ch:i for i, ch in enumerate(chars)}
int_to_string = {i:ch for i, ch in enumerate(chars)}
encode = lambda s:[string_to_int[c] for c in s]
decode = lambda l:''.join([int_to_string[i] for i in l]) 

data = torch.tensor(encode(text), dtype=torch.long)

In [ ]:
n = int(0.8*len(data))
train_data=data[:n]
val_data=data[n:]                                                                                                                                                       

In [12]:
block_size=8
x=train_data[:block_size]
y=train_data[1:block_size+1]
for t in range(block_size):
    context=x[:t+1]
    target=y[t]
    print('When input is ',context,'Output is ',target)

When input is  tensor([80]) Output is  tensor(1)
When input is  tensor([80,  1]) Output is  tensor(1)
When input is  tensor([80,  1,  1]) Output is  tensor(28)
When input is  tensor([80,  1,  1, 28]) Output is  tensor(39)
When input is  tensor([80,  1,  1, 28, 39]) Output is  tensor(42)
When input is  tensor([80,  1,  1, 28, 39, 42]) Output is  tensor(39)
When input is  tensor([80,  1,  1, 28, 39, 42, 39]) Output is  tensor(44)
When input is  tensor([80,  1,  1, 28, 39, 42, 39, 44]) Output is  tensor(32)
